In [2]:
import numpy as np
import pandas as pd
import random
import os

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from stable_baselines3.common.vec_env import DummyVecEnv

from env import StockLearningEnv
import config

In [3]:
df = pd.read_csv('./data_file/train.csv')

In [4]:
e_train_gym = StockLearningEnv(df = df, **config.ENV_PARAMS)

加载数据缓存
数据缓存成功!


In [5]:
from stable_baselines3 import DDPG
from stable_baselines3 import A2C
from stable_baselines3 import PPO
from stable_baselines3 import TD3
from stable_baselines3 import SAC

import pandas as pd
import numpy as np
import config

In [6]:
episode = 50000
model = DDPG(policy='MlpPolicy', env=e_train_gym, **config.DDPG_PARAMS)
model.learn(total_timesteps= episode)
model.save(os.path.join('train_file', "{}.model".format('DDPG' + str(episode))))

EPISODE|STEPS|TERMINAL_REASON|CASH           |TOT_ASSETS     |TERMINAL_REWARD|GAINLOSS_PCT|RETREAT_PROPORTION
   0| 499|update         |￥26,549        |￥2,276,724     |113.61772%|127.67236%|-14.05%   
   0| 999|update         |￥51,781        |￥1,741,219     |39.85222% |74.12192% |-34.27%   
   0|1368|Last Date      |￥6,388         |￥1,690,649     |32.76649% |69.06486% |-36.30%   
   1| 499|update         |￥67,210        |￥1,187,003     |16.73394% |18.70030% |-1.97%    
   1| 999|update         |￥67,210        |￥1,806,196     |53.17972% |80.61961% |-27.44%   
   1|1499|update         |￥67,210        |￥2,772,930     |174.45440%|177.29296%|-2.84%    
   1|1740|Last Date      |￥67,210        |￥2,315,017     |108.28290%|131.50172%|-23.22%   


In [23]:
trade_df = pd.read_csv('./data_file/trade.csv')
e_trade_gym = StockLearningEnv(df = trade_df, **config.ENV_TRADE_PARAMS)

加载数据缓存
数据缓存成功!


In [19]:
episode = 50000
model = PPO(policy='MlpPolicy', env=e_trade_gym, **config.PPO_PARAMS)
model.load(os.path.join('train_file', "{}.model".format('PPO' + str(episode))))

In [35]:
from tqdm import tqdm

test_env, test_obs = e_trade_gym.get_sb_env()


test_env.reset()

len_environment = len(e_trade_gym.df.index.unique())
for i in range(len_environment):
    action, _states = model.predict(test_obs)
    test_obs, _, dones, _ = test_env.step(action)
    if i == len_environment -2:
        df_account = test_env.env_method(method_name="save_asset_memory")[0]
        df_action = test_env.env_method(method_name="save_action_memory")[0]
        print("回测完成!")

EPISODE|STEPS|TERMINAL_REASON|CASH           |TOT_ASSETS     |TERMINAL_REWARD|GAINLOSS_PCT|RETREAT_PROPORTION
回测完成!
   1| 486|Last Date      |￥69,208        |￥1,539,112     |47.95132% |53.91116% |-5.96%    


In [39]:
model_name = 'PPO'
df_action.to_csv('./backtest/' + model_name + 'action.csv')

In [40]:
df_account.to_csv('./backtest/' + model_name + 'account.csv')

In [ ]:
class Policy(nn.Module):
    def __init__(self, in_dim, n_hidden_1, n_hidden_2, num_outputs):
        super(Policy, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(in_dim, n_hidden_1),
            nn.ReLU(True),
            nn.Linear(n_hidden_1, n_hidden_2),
            nn.ReLU(True),
            nn.Linear(n_hidden_2, num_outputs)
        )

class Normal(nn.Module):
    def __init__(self, num_outputs):
        super().__init__()
        self.stds = nn.Parameter(torch.zeros(num_outputs))
    def forward(self, x):
        dist = torch.distributions.Normal(loc=x, scale=self.stds.exp())
        action = dist.sample()
        return action

if __name__ == '__main__':
    policy = Policy(4,20,20,5)
    normal = Normal(5)
    observation = torch.Tensor(4)
    action = normal.forward(policy.layer( observation))
    print("action: ",action)